In [1]:
from dataset import Dataset, Synthetic_data

import numpy as np
from scipy.linalg import orthogonal_procrustes

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tqdm import tqdm

from mu_dist import Mu
from sigma_dist import Sigma_Star
from gamma_dist import Gamma

from r_dist import R
from z_dist import Z
from phi_dist import Phi

from dataset_initialisation import GMM_Init

from scipy.stats import beta

In [2]:
μ_1 = np.array([0.5,0.25,0.25])
μ_2 = np.array([0.4,0.15,0.45])

    # [0.8018,0.2673,0.5345] 
    # [0.2673,0.8018,0.5345]
α = 2
β = 2
prior = lambda : beta.rvs(α, β)

In [3]:
from dataset_initialisation import GMM_Init

In [4]:
ds = Synthetic_data(μ_1, μ_2, prior, N_t=1000)

In [9]:
ds.gaussian_mm_init()

[[0.15157955 0.07850044 0.12538365]
 [0.3157091  0.13092938 0.22855135]] cluster centres
self.gamma_estimates:  [[ 0.01513975  0.01279655]
 [-0.07422802 -0.0333508 ]]
scale_mat gmm [[ 0.03210903 -0.01228337]
 [-0.01228337  0.05878728]]
dof gmm 6
mu_cov 0 det: -2.5332852358956764e-39
scale_mat 0 det: 0.0017367213178696881
gamma_cov 0 det: 0.0
gamma cov: [[0.0039933  0.00206204]
 [0.00206204 0.00106479]]
mu_cov 1 det: -2.5332852358956764e-39
scale_mat 1 det: 0.0017367213178696881
gamma_cov 1 det: 0.0
gamma cov: [[0.0039933  0.00206204]
 [0.00206204 0.00106479]]


In [5]:
def print_attributes(obj):
    attributes = dir(obj)
    attributes = [attr for attr in attributes if not attr.startswith('__') and not callable(getattr(obj, attr))]
    for attr in attributes:
        value = getattr(obj, attr)
        print(f"{attr}: {value}")

## μ variables

In [12]:
print_attributes(ds.means_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.01346925 0.00430257 0.00846643]
 [0.00430257 0.0013744  0.00270449]
 [0.00846643 0.00270449 0.00532179]]
d: 3
k: 0
mean: [0.15157955 0.07850044 0.12538365]
prior_cov: [[0.01346925 0.00430257 0.00846643]
 [0.00430257 0.0013744  0.00270449]
 [0.00846643 0.00270449 0.00532179]]
prior_mean: [0. 0. 0.]

cov matrix det: -2.5332852358956764e-39


In [14]:
print_attributes(ds.means_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.01346925 0.00430257 0.00846643]
 [0.00430257 0.0013744  0.00270449]
 [0.00846643 0.00270449 0.00532179]]
d: 3
k: 1
mean: [0.3157091  0.13092938 0.22855135]
prior_cov: [[0.01346925 0.00430257 0.00846643]
 [0.00430257 0.0013744  0.00270449]
 [0.00846643 0.00270449 0.00532179]]
prior_mean: [0. 0. 0.]

cov matrix det: -2.5332852358956764e-39


## γ variables

In [15]:
print_attributes(ds.gamma_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[0].cov))

cov: [[0.0039933  0.00206204]
 [0.00206204 0.00106479]]
d: 3
dim: 2
k: 0
mean: [0.01513975 0.01279655]
prior_cov: [[0.0039933  0.00206204]
 [0.00206204 0.00106479]]

cov matrix det: 0.0


In [16]:
print_attributes(ds.gamma_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[1].cov))

cov: [[0.0039933  0.00206204]
 [0.00206204 0.00106479]]
d: 3
dim: 2
k: 1
mean: [-0.07422802 -0.0333508 ]
prior_cov: [[0.0039933  0.00206204]
 [0.00206204 0.00106479]]

cov matrix det: 0.0


## sigma star variables

In [19]:
print_attributes(ds.sigma_star_vars[0])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[0].scale))

d: 3
dim: 2
dof: 6
first_moment: None
k: 0
prior_dof: 6
prior_scale: [[ 0.03210903 -0.01228337]
 [-0.01228337  0.05878728]]
scale: [[0.01961113 0.00189058]
 [0.00189058 0.02104674]]
sigma_star_scale_det: 0.00040917606877306176


In [20]:
print_attributes(ds.sigma_star_vars[1])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[1].scale))


d: 3
dim: 2
dof: 6
first_moment: None
k: 1
prior_dof: 6
prior_scale: [[ 0.03210903 -0.01228337]
 [-0.01228337  0.05878728]]
scale: [[ 0.04460693 -0.02645732]
 [-0.02645732  0.09652781]]
sigma_star_scale_det: 0.0036058196618045115


In [21]:
ds.dataset_vi()

[[0.14944866 0.07575788 0.1202365 ]
 [0.309299   0.13102468 0.22847952]] cluster centres
self.gamma_estimates:  [[ 0.01478469  0.01209391]
 [-0.07203667 -0.03300367]]
scale_mat gmm [[ 0.03249915 -0.01175278]
 [-0.01175278  0.05658144]]
dof gmm 6
mu_cov 0 det: 0.0
scale_mat 0 det: 0.0017007209536410545
gamma_cov 0 det: -8.172659755515827e-22
gamma cov: [[0.00376897 0.00195772]
 [0.00195772 0.0010169 ]]
mu_cov 1 det: 0.0
scale_mat 1 det: 0.0017007209536410545
gamma_cov 1 det: -8.172659755515827e-22
gamma cov: [[0.00376897 0.00195772]
 [0.00195772 0.0010169 ]]


Performing VI:   0%|          | 0/1000 [00:00<?, ?it/s]


LinAlgError: Singular matrix